## Trinomial Search For Documents Stored in Constellate's Datasets
This Jupyter Notebook contains programming to search for trinomial instances in datasets built using Constellate's dataset builder. 

**How this notebook functions:** ADD TEXT HERE

**What you need to start using this notebook:**
